In [1]:
import json
import minsearch
import tqdm
import requests 

from openai import OpenAI

from elasticsearch import Elasticsearch

# Q1

In [2]:
!curl localhost:9200

{
  "name" : "4f22981f244f",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "NBP7vqsMSTKYAEgzTfqaeQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


# Q2

index

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Q3

In [18]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-homework_w1"

In [19]:
es = Elasticsearch('http://localhost:9200') 

In [20]:
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework_w1'})

In [24]:
for doc in documents:
    es.index(index=index_name, body=doc)

In [27]:
query = "How do I execute a command in a running docker container?"
search_query = {
    "size": 5,
    "query": {
        "bool":{
            "must":{
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [28]:
response = es.search(index=index_name, body=search_query)
print(response['hits']['hits'][0]['_score'])

84.050095


# Q4

In [29]:
search_query = {
    "size": 3,
    "query": {
        "bool":{
            "must":{
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [31]:
response = es.search(index=index_name, body=search_query)
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5

In [32]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool":{
                "must":{
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es.search(index=index_name, body=search_query)
    resp = []
    for hit in response['hits']['hits']:
        resp.append(hit['_source'])

    return resp

In [39]:
def build_prompt(query, context):
    prompt_temp = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    prompt = prompt_temp.format(question=query, context=context)
    return prompt

In [40]:
resp_doc = elastic_search(query)

In [41]:
context = ""
context_tmp= """
Q: {question}
A: {text}
""".strip()

for doc in resp_doc:
    context += (context_tmp.format(question=doc['question'], text=doc['text']))

context = context.strip()

In [42]:
prompt = build_prompt(query, context)
len(prompt)

1458

# Q6

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")
tokens = encoding.encode(prompt)
print(len(tokens))
